# GDACS Data with pystac-monty

This notebook demonstrates how to use pystac-monty to process GDACS (Global Disaster Alert and Coordination System) data and visualize it using interactive maps. We'll:

1. Download GDACS data from the GDACS API
2. Convert the data to STAC items using pystac-monty
3. Display events on an interactive map
4. Explore the Monty STAC model and its metadata

Let's begin by importing the necessary libraries.

In [1]:
%load_ext autoreload
%autoreload 2

# Basic libraries
import tempfile
import json

# Visualization libraries
import folium
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import requests

# STAC and pystac-monty
import shapely
from folium.plugins import MarkerCluster
from IPython.display import clear_output, display

from pystac_monty.extension import MontyExtension
from pystac_monty.geocoding import WorldAdministrativeBoundariesGeocoder
from pystac_monty.sources.gdacs import GDACSDataSource, GDACSDataSourceType, GDACSTransformer

## 1. Download and Process GDACS Data

First, let's get a list of GDACS events and then download detailed data for a specific event.

In [6]:
# Define GDACS API parameters for event list
event_type = ""  # Flood events
from_date = "01-01-2020"  # Start date
to_date = "01-01-2021"    # End date
country = ""           # Spain (optional)
limit = 1000                # Limit the number of events

# Construct the GDACS API URL for event list
gdacs_event_list_url = f"https://www.gdacs.org/gdacsapi/api/events/geteventlist/SEARCH?eventtype={event_type}&fromdate={from_date}&todate={to_date}&limit={limit}"
if country:
    gdacs_event_list_url += f"&country={country}"

print(f"Downloading GDACS event list from {gdacs_event_list_url}...")
event_list_response = requests.get(gdacs_event_list_url)
event_list_data = event_list_response.text
print(f"Downloaded {len(event_list_data)} bytes of event list data")

# Parse the event list data
event_list_json = json.loads(event_list_data)
print(f"Number of events in the list: {len(event_list_json.get('features', []))}")

# Initialize the geocoder
geocoder = WorldAdministrativeBoundariesGeocoder("../tests/data-files/world-administrative-boundaries.fgb")

Downloaded 103542 bytes of event list data
Number of events in the list: 70


## 2. Create STAC Items from GDACS Data

Now, let's transform the GDACS data into STAC items.

In [ ]:

all_stac_items = []

# loop on events and episodes
for selected_event in event_list_json.get('features', []):
    event_id = selected_event.get('properties', {}).get('eventid')
    event_type = selected_event.get('properties', {}).get('eventtype')
    episode_id = selected_event.get('properties', {}).get('episodeid')
    
    # get all episodes of the event
    for episode in range(1, episode_id):
        print(f"\nSelected event: ID={event_id}, Type={event_type}, Episode={episode}")
        
        # Construct the GDACS API URLs for detailed event data
        # e.g. https://www.gdacs.org/gdacsapi/api/events/geteventdata?eventtype=FL&eventid=1102983`
        #      https://www.gdacs.org/gdacsapi/api/polygons/getgeometry?eventtype=FL&eventid=1102983&episodeid=2
        gdacs_event_url = f"https://www.gdacs.org/gdacsapi/api/events/geteventdata?eventtype={event_type}&eventid={event_id}"
        gdacs_geometry_url = f"https://www.gdacs.org/gdacsapi/api/polygons/getgeometry?eventtype={event_type}&eventid={event_id}&episodeid={episode}"
        
        print(f"\nDownloading GDACS event data from {gdacs_event_url}...")
        event_response = requests.get(gdacs_event_url)
        event_data = event_response.text
        print(f"Downloaded {len(event_data)} bytes of event data")
        
        print(f"Downloading GDACS geometry data from {gdacs_geometry_url}...")
        geometry_response = requests.get(gdacs_geometry_url)
        geometry_data = geometry_response.text
        print(f"Downloaded {len(geometry_data)} bytes of geometry data")
        
        # Initialize the data sources
        event_data_source = GDACSDataSource(gdacs_event_url, event_data, GDACSDataSourceType.EVENT)
        geometry_data_source = None
        if geometry_data and len(geometry_data) > 0:
            geometry_data_source = GDACSDataSource(gdacs_geometry_url, geometry_data, GDACSDataSourceType.GEOMETRY)
        
        # Initialize the transformer with both data sources
        transformer = GDACSTransformer([event_data_source, geometry_data_source])

        # Create STAC items
        print("Creating STAC items from GDACS data...")
        stac_items = transformer.make_items()
        all_stac_items.extend(stac_items)
        print(f"Created {len(stac_items)} STAC items")


Selected event: ID=1014149, Type=DR, Episode=1

Downloaded 3191 bytes of event data
Downloaded 3432 bytes of geometry data
Creating STAC items from GDACS data...
Created 2 STAC items

Selected event: ID=1014149, Type=DR, Episode=2

Downloaded 3191 bytes of event data
Downloaded 3163 bytes of geometry data
Creating STAC items from GDACS data...
Created 2 STAC items

Selected event: ID=1014149, Type=DR, Episode=3

Downloaded 3191 bytes of event data
Downloaded 3163 bytes of geometry data
Creating STAC items from GDACS data...
Created 2 STAC items

Selected event: ID=1014149, Type=DR, Episode=4

Downloaded 3191 bytes of event data
Downloaded 3163 bytes of geometry data
Creating STAC items from GDACS data...
Created 2 STAC items

Selected event: ID=1014149, Type=DR, Episode=5

Downloaded 3191 bytes of event data
Downloaded 3161 bytes of geometry data
Creating STAC items from GDACS data...
Created 2 STAC items

Selected event: ID=1014149, Type=DR, Episode=6

Downloaded 3191 bytes of event 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
# Separate the STAC items by role
event_items = []
hazard_items = []
impact_items = []

for item in all_stac_items:
    roles = item.properties.get("roles", [])
    if "event" in roles:
        event_items.append(item)
    elif "hazard" in roles:
        hazard_items.append(item)
    elif "impact" in roles:
        impact_items.append(item)

print(f"Events: {len(event_items)}, Hazards: {len(hazard_items)}, Impacts: {len(impact_items)}")

Events: 231, Hazards: 231, Impacts: 0


## 3. Add Items to STAC Collections

Now, let's add the STAC items to their respective collections in a STAC API.

In [9]:
# Import STAC helper functions
import sys
sys.path.append('.')
from stac_helpers import (
    add_items_to_collection,
    check_collection_exists,
    check_stac_api_availability,
    create_collection_fallback,
    create_collection_from_file,
    delete_collection,
)

In [10]:
# Define the STAC API endpoint
# Replace with your actual STAC API endpoint
stac_api_url = "https://montandon-eoapi-stage.ifrc.org/stac"

# Define the collection IDs for each type of item
# These match the predefined collections in monty-stac-extension/examples
event_collection_id = "gdacs-events"
hazard_collection_id = "gdacs-hazards"
impact_collection_id = "gdacs-impacts"

# Define paths to the predefined collection definitions
event_collection_path = "../monty-stac-extension/examples/gdacs-events/gdacs-events.json"
hazard_collection_path = "../monty-stac-extension/examples/gdacs-hazards/gdacs-hazards.json"
impact_collection_path = "../monty-stac-extension/examples/gdacs-impacts/gdacs-impacts.json"

# Check if the STAC API is available
api_available = check_stac_api_availability(stac_api_url)

STAC API is available at https://montandon-eoapi-stage.ifrc.org/stac


In [11]:
# Check if the collections exist and create them if they don't
if api_available:
    # Check and create event collection if needed
    delete_collection(stac_api_url, event_collection_id)
    event_collection_exists = check_collection_exists(stac_api_url, event_collection_id)
    if not event_collection_exists:
        print(f"\nAttempting to create collection '{event_collection_id}'...")
        event_collection_created = create_collection_from_file(stac_api_url, event_collection_path)
        if not event_collection_created:
            print("Trying fallback method to create event collection...")
            event_collection_created = create_collection_fallback(
                stac_api_url, 
                event_collection_id, 
                "GDACS events processed with pystac-monty",
                ["event", "source"]
            )
        event_collection_exists = event_collection_created
    
    # Check and create hazard collection if needed
    delete_collection(stac_api_url, hazard_collection_id)
    hazard_collection_exists = check_collection_exists(stac_api_url, hazard_collection_id)
    if not hazard_collection_exists:
        print(f"\nAttempting to create collection '{hazard_collection_id}'...")
        hazard_collection_created = create_collection_from_file(stac_api_url, hazard_collection_path)
        if not hazard_collection_created:
            print("Trying fallback method to create hazard collection...")
            hazard_collection_created = create_collection_fallback(
                stac_api_url, 
                hazard_collection_id, 
                "GDACS hazards processed with pystac-monty",
                ["hazard", "source"]
            )
        hazard_collection_exists = hazard_collection_created
    
    # Check and create impact collection if needed
    if impact_items:
        delete_collection(stac_api_url, impact_collection_id)
        impact_collection_exists = check_collection_exists(stac_api_url, impact_collection_id)
        if not impact_collection_exists:
            print(f"\nAttempting to create collection '{impact_collection_id}'...")
            impact_collection_created = create_collection_from_file(stac_api_url, impact_collection_path)
            if not impact_collection_created:
                print("Trying fallback method to create impact collection...")
                impact_collection_created = create_collection_fallback(
                    stac_api_url, 
                    impact_collection_id, 
                    "GDACS impacts processed with pystac-monty",
                    ["impact", "source"]
                )
            impact_collection_exists = impact_collection_created
    else:
        impact_collection_exists = False
    
    if not (event_collection_exists and hazard_collection_exists and (not impact_items or impact_collection_exists)):
        print("\nWarning: One or more collections could not be created in the STAC API.")
        print("Some items may not be added to the STAC API.")
else:
    print("STAC API is not available. Skipping collection checks and creation.")

Collection 'gdacs-events' does not exist in the STAC API
Collection 'gdacs-events' does not exist, nothing to delete
Collection 'gdacs-events' does not exist in the STAC API

Attempting to create collection 'gdacs-events'...
Collection 'gdacs-events' created successfully
Collection 'gdacs-hazards' does not exist in the STAC API
Collection 'gdacs-hazards' does not exist, nothing to delete
Collection 'gdacs-hazards' does not exist in the STAC API

Attempting to create collection 'gdacs-hazards'...
Collection 'gdacs-hazards' created successfully


In [15]:
# Add the items to their respective collections if the API is available
if api_available:
    if event_collection_exists:
        print("Adding event items to the collection...")
        event_success, event_failed = add_items_to_collection(stac_api_url, event_collection_id, event_items, overwrite=True)
    else:
        print("Skipping adding event items because the collection doesn't exist")
        event_success, event_failed = 0, len(event_items)
    
    if hazard_collection_exists:
        print("\nAdding hazard items to the collection...")
        hazard_success, hazard_failed = add_items_to_collection(stac_api_url, hazard_collection_id, hazard_items)
    else:
        print("Skipping adding hazard items because the collection doesn't exist")
        hazard_success, hazard_failed = 0, len(hazard_items)
    
    if impact_items and impact_collection_exists:
        print("\nAdding impact items to the collection...")
        impact_success, impact_failed = add_items_to_collection(stac_api_url, impact_collection_id, impact_items)
    else:
        print("Skipping adding impact items because the collection doesn't exist or there are no impact items")
        impact_success, impact_failed = 0, len(impact_items)
    
    total_success = event_success + hazard_success + impact_success
    total_failed = event_failed + hazard_failed + impact_failed
    
    print(f"\nSummary: Added {total_success} items successfully, {total_failed} items failed")
else:
    print("Skipping adding items to collections because the API is not available")

Adding event items to the collection...
Processing batch 1 of 1 (231 items)
Added 0 items successfully, 0 items failed

Adding hazard items to the collection...
Processing batch 1 of 1 (231 items)
Added 0 items successfully, 0 items failed
Skipping adding impact items because the collection doesn't exist or there are no impact items

Summary: Added 0 items successfully, 0 items failed


## 4. Visualize GDACS Events on a Map

Let's create an interactive map to visualize the GDACS events.

In [ ]:
# Create a map centered on the first event's location or a default location
if event_items:
    # Get the coordinates of the first event
    first_event = event_items[0]
    coords = first_event.geometry["coordinates"]
    center = [coords[1], coords[0]]  # [lat, lon]
else:
    # Default center (Madrid, Spain)
    center = [40.4168, -3.7038]

# Create the map
m = folium.Map(location=center, zoom_start=6)

# Add a marker cluster for the events
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each event
for item in event_items:
    # Get the coordinates
    coords = item.geometry["coordinates"]
    lat, lon = coords[1], coords[0]
    
    # Get event properties
    title = item.properties.get("title", "No title")
    description = item.properties.get("description", "No description")
    event_type = item.properties.get("monty:hazard_codes", ["Unknown"])[0]
    date = item.datetime.strftime("%Y-%m-%d")
    
    # Create popup content
    popup_content = f"<b>ID:</b> {item.id}<br>"
    popup_content += f"<b>Type:</b> {event_type}<br>"
    popup_content += f"<b>Date:</b> {date}<br>"
    popup_content += f"<b>Title:</b> {title}<br>"
    popup_content += f"<b>Description:</b> {description[:200]}...<br>"
    
    # Add marker to the cluster
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_content, max_width=300),
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(marker_cluster)

# Display the map
m